In [1]:
from experiment import Experiment
%matplotlib inline

In [2]:
var_dict = dict(
    categorical_vars = ['Work_accident', 'promotion_last_5years', 'sales', 
                        'salary', 'number_project','time_spend_company'],
    numerical_vars = ['satisfaction_level', 'last_evaluation', 'average_montly_hours'],
    class_var = 'left'
)

In [3]:
exp = Experiment('data/HR_comma_sep.csv', 
                 var_dict,
                 embedding_dim=16,
                 n_epoch=1,
                 verbose=True,
                 n_bins_range=range(2, 21),
                 n_init_bins_list=[5, 10, 15, 20])

In [4]:
%time clf_scores = exp.test_classification_performance(cv=3)

>>> Epoch = 1, Loss = 3.826228380203247
Learning Embedding Finished!
>>> Epoch = 1, Loss = 4.066385269165039
Learning Embedding Finished!
>>> Epoch = 1, Loss = 4.265279293060303
Learning Embedding Finished!
>>> Epoch = 1, Loss = 4.522012233734131
Learning Embedding Finished!
>>> Epoch = 1, Loss = 3.9101059436798096
Learning Embedding Finished!
>>> Epoch = 1, Loss = 4.202967166900635
Learning Embedding Finished!
>>> Epoch = 1, Loss = 4.362939834594727
Learning Embedding Finished!
>>> Epoch = 1, Loss = 4.564137935638428
Learning Embedding Finished!
>>> Epoch = 1, Loss = 3.7820827960968018
Learning Embedding Finished!
>>> Epoch = 1, Loss = 4.178693771362305
Learning Embedding Finished!
>>> Epoch = 1, Loss = 4.312819957733154
Learning Embedding Finished!
>>> Epoch = 1, Loss = 4.579192161560059
Learning Embedding Finished!
CPU times: user 58 s, sys: 1min 28s, total: 2min 26s
Wall time: 47.9 s


In [5]:
exp.print_classification_scores(clf_scores, method='dummy_only')

DT performance
#cols = 34.0, Accuracy = 0.959 (+/- 0.003)
LR performance
#cols = 34.0, Accuracy = 0.896 (+/- 0.008)


In [6]:
exp.print_classification_scores(clf_scores, method='scale_numeric')

DT performance
#cols = 34.0, Accuracy = 0.959 (+/- 0.003)
LR performance
#cols = 34.0, Accuracy = 0.896 (+/- 0.008)


In [7]:
exp.print_classification_scores(clf_scores, method='equal_width')

DT performance
#Bins = 2, #Avg Cols = 37.0, Accuracy = 0.941 (+/- 0.005)
#Bins = 3, #Avg Cols = 40.0, Accuracy = 0.949 (+/- 0.006)
#Bins = 4, #Avg Cols = 43.0, Accuracy = 0.941 (+/- 0.002)
#Bins = 5, #Avg Cols = 46.0, Accuracy = 0.937 (+/- 0.008)
#Bins = 6, #Avg Cols = 49.0, Accuracy = 0.944 (+/- 0.002)
#Bins = 7, #Avg Cols = 52.0, Accuracy = 0.943 (+/- 0.009)
#Bins = 8, #Avg Cols = 55.0, Accuracy = 0.937 (+/- 0.003)
#Bins = 9, #Avg Cols = 58.0, Accuracy = 0.938 (+/- 0.003)
#Bins = 10, #Avg Cols = 61.0, Accuracy = 0.929 (+/- 0.006)
#Bins = 11, #Avg Cols = 64.0, Accuracy = 0.939 (+/- 0.013)
#Bins = 12, #Avg Cols = 67.0, Accuracy = 0.945 (+/- 0.015)
#Bins = 13, #Avg Cols = 70.0, Accuracy = 0.930 (+/- 0.014)
#Bins = 14, #Avg Cols = 73.0, Accuracy = 0.941 (+/- 0.015)
#Bins = 15, #Avg Cols = 76.0, Accuracy = 0.941 (+/- 0.012)
#Bins = 16, #Avg Cols = 79.0, Accuracy = 0.932 (+/- 0.006)
#Bins = 17, #Avg Cols = 82.0, Accuracy = 0.934 (+/- 0.007)
#Bins = 18, #Avg Cols = 85.0, Accuracy = 0.933 (+

In [8]:
exp.print_classification_scores(clf_scores, method='equal_freq')

DT performance
#Bins = 2, #Avg Cols = 37.0, Accuracy = 0.929 (+/- 0.013)
#Bins = 3, #Avg Cols = 40.0, Accuracy = 0.945 (+/- 0.005)
#Bins = 4, #Avg Cols = 43.0, Accuracy = 0.926 (+/- 0.008)
#Bins = 5, #Avg Cols = 46.0, Accuracy = 0.942 (+/- 0.001)
#Bins = 6, #Avg Cols = 49.0, Accuracy = 0.943 (+/- 0.004)
#Bins = 7, #Avg Cols = 52.0, Accuracy = 0.938 (+/- 0.004)
#Bins = 8, #Avg Cols = 55.0, Accuracy = 0.939 (+/- 0.008)
#Bins = 9, #Avg Cols = 58.0, Accuracy = 0.937 (+/- 0.006)
#Bins = 10, #Avg Cols = 61.0, Accuracy = 0.931 (+/- 0.007)
#Bins = 11, #Avg Cols = 64.0, Accuracy = 0.926 (+/- 0.001)
#Bins = 12, #Avg Cols = 67.0, Accuracy = 0.931 (+/- 0.012)
#Bins = 13, #Avg Cols = 70.0, Accuracy = 0.935 (+/- 0.015)
#Bins = 14, #Avg Cols = 73.0, Accuracy = 0.937 (+/- 0.009)
#Bins = 15, #Avg Cols = 76.0, Accuracy = 0.936 (+/- 0.007)
#Bins = 16, #Avg Cols = 79.0, Accuracy = 0.943 (+/- 0.007)
#Bins = 17, #Avg Cols = 82.0, Accuracy = 0.946 (+/- 0.003)
#Bins = 18, #Avg Cols = 85.0, Accuracy = 0.944 (+

In [9]:
exp.print_classification_scores(clf_scores, method='semantic_binning')

DT performance
#Init Bins = 5, #Avg Cols = 39.0, Accuracy = 0.932 (+/- 0.005)
#Init Bins = 10, #Avg Cols = 48.0, Accuracy = 0.941 (+/- 0.009)
#Init Bins = 15, #Avg Cols = 58.333333333333336, Accuracy = 0.940 (+/- 0.008)
#Init Bins = 20, #Avg Cols = 72.33333333333333, Accuracy = 0.936 (+/- 0.016)
LR performance
#Init Bins = 5, #Avg Cols = 39.0, Accuracy = 0.894 (+/- 0.003)
#Init Bins = 10, #Avg Cols = 48.0, Accuracy = 0.917 (+/- 0.019)
#Init Bins = 15, #Avg Cols = 58.333333333333336, Accuracy = 0.934 (+/- 0.028)
#Init Bins = 20, #Avg Cols = 72.33333333333333, Accuracy = 0.943 (+/- 0.023)
NB performance
#Init Bins = 5, #Avg Cols = 39.0, Accuracy = 0.858 (+/- 0.006)
#Init Bins = 10, #Avg Cols = 48.0, Accuracy = 0.884 (+/- 0.026)
#Init Bins = 15, #Avg Cols = 58.333333333333336, Accuracy = 0.898 (+/- 0.032)
#Init Bins = 20, #Avg Cols = 72.33333333333333, Accuracy = 0.910 (+/- 0.040)


### 학습된 트리의 깊이, 형태 비교

In [ ]:
dot_data = StringIO()
export_graphviz(dt, out_file=dot_data, feature_names=X.columns, 
                impurity=False, proportion=True)
graph = pydot.graph_from_dot_data(dot_data.getvalue())